In [54]:
from tensorflow.keras.models import load_model
import joblib 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def model_wine(df):

    X = df.drop('quality', axis=1)
    y = df['quality']

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalise: each column has mean 0 and standard deviation 1, so that the model can learn more efficiently and stably
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    #sequential is a type of network where each layer goes directly after the previous one (ideal for simple architectures)
    model = Sequential([
        #Dense creates a dense layer, it means its fully conected (each neuron is connected with all of the previous layer)
        #Dense(number of neurons, each neuron uses the activation function ReLU (Rectified Linear Unit), which introduces
        #  no-linearity, helpful for complex patterns)
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # just one output
    ])

    # configures how model is gonna be trained
    model.compile(optimizer='adam',# adam is a very famous algorithm of weights optimization
        loss='mean_squared_error',# This is the loss function(the measure the model tries to minimize),
                                # MSE isIdeal for continuos regression problems like this 
        metrics=['mae'])#set metrics used for evaluating the training, mean absolute error is easy to interpret


    #train
    history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)


    #evaluate
    loss, mae = model.evaluate(X_test, y_test)
    print(f"MAE:{mae:.2f} This tells you, on average, the model is predicting the wine quality within {mae:.2f} points on a 1–10 scale") 
    model.save('wine_quality_model.keras')
    joblib.dump(scaler, 'scaler.pkl')

In [55]:
# Clean and save multiple datasets
df1=pd.read_csv("../cleanDatasets/winequality-red_cleaned.csv", sep=",", quotechar='"')
df2=pd.read_csv("../cleanDatasets/winequality-white_cleaned.csv", sep=",", quotechar='"')

In [61]:
#model_wine(df1)
model_wine(df2)

Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 20.9640 - mae: 4.0937 - val_loss: 2.7783 - val_mae: 1.2221
Epoch 2/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - loss: 2.3310 - mae: 1.1992 - val_loss: 2.0238 - val_mae: 1.0221
Epoch 3/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: 1.5778 - mae: 0.9736 - val_loss: 1.4480 - val_mae: 0.8737
Epoch 4/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - loss: 1.1391 - mae: 0.8383 - val_loss: 1.1208 - val_mae: 0.7661
Epoch 5/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - loss: 0.7991 - mae: 0.6934 - val_loss: 0.8742 - val_mae: 0.6872
Epoch 6/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - loss: 0.6598 - mae: 0.6317 - val_loss: 0.6712 - val_mae: 0.6085
Epoch 7/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 0.6076 - mae: 0.6042 - val_loss: 0.6226 - val_mae: 0.6054
Epoch 8/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 0.5384 - mae: 0.5722 - val_loss: 0.5969 - val_mae: 0.5931
Epoch 9/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 7

In [62]:
wine_model = load_model('wine_quality_model.keras')
scaler = joblib.load('scaler.pkl')

In [63]:
sample = df2.drop('quality', axis=1).sample(1, random_state=42)

In [64]:
sample_scaled = scaler.transform(sample)
predicted_quality = wine_model.predict(sample_scaled)

print(f"\nInput data:\n{sample}")
print(f"\nPredicted wine quality (1–10 scale): {predicted_quality[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Input data:
     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
149            7.2              0.32         0.47             5.1      0.044   

     free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
149                 19.0                  65.0    0.991  3.03       0.41   

     alcohol  
149     12.6  

Predicted wine quality (1–10 scale): 6.14
